# Computing gradients in tensorflow

## Partial derivatives using pure python

Let's say we have a derivable function:

\begin{equation}
f(x) = 2 x_1^2 + 3 x_1 x_2
\end{equation}

In [ ]:
def f(x1, x2):
    return 2 * x1 ** 2 + 3 * x1 * x2

It's easy to find analytically the derivative of this function:

\begin{align}
\frac{\partial{f}}{\partial x_1} & = 4x_1 + 3x_2\\
\frac{\partial{f}}{\partial x_2} & = 3x_1 
\end{align}

So, for the point $x=(2,1)$, the result will be $(11,6)$. To check that everything goes as expected, we can compute the partial derivatives with regard to both variables using the definition:

\begin{equation}
\frac{\partial}{{\partial x}}f \left( x \right) = \mathop {\lim }\limits_{\epsilon \to 0} \frac{{f\left( {x + \epsilon } \right) - f\left( x \right)}}{\epsilon }
\end{equation}

In [ ]:
x1, x2 = 2, 1
eps = 1e-04

In [ ]:
(f(x1 + eps, x2) - f(x1, x2)) / (eps)

In [ ]:
(f(x1, x2 + eps) - f(x1, x2)) / (eps)

Great! So far so good.

## Partial derivatives using tensorflow

We will do the same, but this time we will use tensorlfow to calculate the results. It may not be as interesting, but it will certainly be more efficient.

In [ ]:
import tensorflow as tf

In [ ]:
x1, x2 = tf.Variable(2.), tf.Variable(1.)

In [ ]:
with tf.GradientTape() as tape:
    y = f(x1, x2)
gradients = tape.gradient(y, [x1, x2])

In [ ]:
[g.numpy() for g in gradients]

Within the `tf.GradientTape` context, tensorflow will track each operation applied to any variable. But be careful! To save memory, tensorflow will remove the tape contents after calling the `.gradient()` method. To avoid this, you can explicitly indicate that you do not want them to disappear (with the `persistent=True` parameter of the `GradientTape`), but try not to do so if there is no good reason, or even remove it from memory once you've done with it.